
# Projeto: Predição e Insights de Filmes — *The Movies Dataset*

**Disciplina:** Fundamentos de Big Data  
**Notebook:** Pipeline completo (bronze → silver → gold), EDA e baseline de modelagem  
**Base:** `movies_metadata.csv` (Kaggle — The Movies Dataset)  



## Objetivos
- Montar um **pipeline de dados** de ponta a ponta cobrindo **Ingestão**, **Transformação**, **Carregamento** e **Destino**.
- Padronizar o schema do `movies_metadata.csv`, tratar erros e enriquecer com colunas derivadas.
- Entregar **arquivos Parquet** nas camadas **/bronze**, **/silver** e **/gold**.
- Realizar **EDA** enxuta e **modelagem baseline** (prever `vote_average` ou `revenue`).
- Salvar **insumos finais** (tabelas e gráficos) para consumo.



## Estrutura de Projeto (sugerida para o repositório)
```
/codigo               # scripts python e notebooks (.ipynb)
/notebooks            # notebooks intermediários (opcional)
/documentacao         # diagrama de arquitetura, PDFs, etc.
/dados
  /raw                # arquivos originais (não versionar grandes)
  /bronze             # parquet/CSV bruto padronizado
  /silver             # parquet já limpo/enriquecido
  /gold               # datasets finais prontos para análise/modelo
README.md
```



## 0. Setup
Execute para instalar/carregar dependências no Colab.


In [ ]:

# Se estiver no Colab, descomente para instalar libs extras (se necessário):
# !pip install pyarrow fastparquet scikit-learn pandas numpy matplotlib

import os
import json
import ast
import math
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

BASE_DIR = Path.cwd() / "dados"
RAW_DIR = BASE_DIR / "raw"
BRONZE_DIR = BASE_DIR / "bronze"
SILVER_DIR = BASE_DIR / "silver"
GOLD_DIR = BASE_DIR / "gold"

for d in [RAW_DIR, BRONZE_DIR, SILVER_DIR, GOLD_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Estrutura criada em:", BASE_DIR.resolve())


: 


## 1. Ingestão (batch)
Escolha uma das opções abaixo:
1. **Upload direto** do arquivo `movies_metadata.csv` para a pasta `/dados/raw`.
2. **Google Drive**: montar Drive e copiar o CSV para `/dados/raw`.
3. **Kaggle API** (no Colab): configurar `kaggle.json` e baixar o dataset (opcional).

> **Nota:** Este notebook assume **execução em batch** (lotes). Para simular **streaming**, ver a seção opcional ao final.


In [2]:

# === Opção 1: Upload manual ===
# from google.colab import files  # no Colab
# uploaded = files.upload()       # selecione movies_metadata.csv
# for fname in uploaded.keys():
#     os.replace(fname, RAW_DIR / fname)
# print("Arquivos enviados para:", RAW_DIR)

# === Opção 2: Google Drive ===
# from google.colab import drive
# drive.mount('/content/drive')
# !cp "/content/drive/MyDrive/caminho/para/movies_metadata.csv" "{RAW_DIR}"

# === Opção 3: Kaggle API (opcional) ===
# !mkdir -p ~/.kaggle
# with open('/root/.kaggle/kaggle.json', 'w') as f:
#     f.write('{"username":"SEU_USUARIO","key":"SUA_CHAVE"}')
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d rounakbanik/the-movies-dataset -f movies_metadata.csv -p "{RAW_DIR}"
# !unzip -o "{RAW_DIR / 'movies_metadata.csv.zip'}" -d "{RAW_DIR}"

raw_csv = RAW_DIR / "movies_metadata.csv"
print("Esperando arquivo em:", raw_csv.resolve())


Esperando arquivo em: /content/dados/raw/movies_metadata.csv



## 2. Bronze — Padronização mínima do bruto
- Leitura robusta (tratando encoding/delimitador).
- Padronização de nomes de colunas (lowercase, snake_case).
- Salvamento em **Parquet** sem transformar semântica (apenas ajustes mínimos de tipos).


In [3]:

def to_snake(s: str) -> str:
    return (s or "").strip().lower().replace(" ", "_").replace("-", "_")

def read_movies_csv(path: Path) -> pd.DataFrame:
    # Leitura tolerante a erros comuns no dataset
    df = pd.read_csv(path, low_memory=False)
    df.columns = [to_snake(c) for c in df.columns]
    return df

if raw_csv.exists():
    df_raw = read_movies_csv(raw_csv)
    print(df_raw.shape, "linhas, colunas:", len(df_raw.columns))
    df_raw.to_parquet(BRONZE_DIR / "movies_metadata_bronze.parquet", index=False)
    print("Bronze salvo em:", (BRONZE_DIR / "movies_metadata_bronze.parquet").resolve())
else:
    print("Arquivo não encontrado:", raw_csv)


Arquivo não encontrado: /content/dados/raw/movies_metadata.csv



## 3. Silver — Limpeza e Enriquecimento
Tratamentos principais recomendados para `movies_metadata`:
- **Tipos**: `budget`, `revenue`, `popularity`, `runtime`, `vote_average`, `vote_count` → numéricos coerentes.
- **Datas**: `release_date` → `datetime`; criar `release_year`.
- **IDs**: garantir que `id` seja tratável (alguns registros possuem valores inválidos).
- **JSON-likes**: colunas como `genres`, `production_companies` etc. → normalizar para listas/strings.
- **Filtros**: remover registros com datas inválidas, duplicatas e valores claramente defeituosos.
- **Derivadas**: `roi = revenue / budget` (quando `budget > 0`), `vote_density = vote_count / (anos desde lançamento + 1)`.


In [ ]:

def coerce_float(series):
    return pd.to_numeric(series, errors="coerce")

def parse_date_safe(s):
    try:
        return pd.to_datetime(s, errors="coerce")
    except Exception:
        return pd.NaT

def parse_json_like(cell):
    # Muitas colunas vêm como strings de listas de dicts; tentar ast.literal_eval
    if pd.isna(cell) or cell in ("", "[]", "{}"):
        return []
    try:
        obj = ast.literal_eval(cell)
        if isinstance(obj, list):
            return obj
        return []
    except Exception:
        return []

def extract_names(list_of_dicts, key="name"):
    try:
        return [d.get(key) for d in list_of_dicts if isinstance(d, dict) and key in d]
    except Exception:
        return []

bronze_path = BRONZE_DIR / "movies_metadata_bronze.parquet"
if bronze_path.exists():
    df = pd.read_parquet(bronze_path)

    # Tipos numéricos
    for col in ["budget","revenue","popularity","runtime","vote_average","vote_count"]:
        if col in df.columns:
            df[col] = coerce_float(df[col])

    # Datas
    if "release_date" in df.columns:
        df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
        df["release_year"] = df["release_date"].dt.year

    # IDs e títulos
    if "id" in df.columns:
        # Alguns ids são inválidos (ex.: strings com datas). Tornar numérico, onde falhar deixar NaN.
        df["id_numeric"] = pd.to_numeric(df["id"], errors="coerce")
    if "title" in df.columns and "original_title" in df.columns:
        df["title_clean"] = df["title"].fillna(df["original_title"])

    # JSON-like: genres
    if "genres" in df.columns:
        df["genres_list"] = df["genres"].apply(parse_json_like).apply(lambda lst: extract_names(lst, "name"))
        df["genres_str"] = df["genres_list"].apply(lambda lst: ", ".join([x for x in lst if isinstance(x, str)]))

    # Derivadas
    if "budget" in df.columns and "revenue" in df.columns:
        df["roi"] = np.where(df["budget"]>0, df["revenue"]/df["budget"], np.nan)
    if "vote_count" in df.columns and "release_year" in df.columns:
        current_year = datetime.now().year
        df["years_since_release"] = np.where(pd.notna(df["release_year"]), current_year - df["release_year"], np.nan)
        df["vote_density"] = np.where((pd.notna(df["years_since_release"])) & (df["years_since_release"] >= 0),
                                      df["vote_count"] / (df["years_since_release"] + 1), np.nan)

    # Filtros de qualidade básicos
    df = df.drop_duplicates(subset=["id","title_clean"], keep="first")
    # Filtrar datas muito antigas/invalidas se necessário (opcional)
    # df = df[df["release_year"].between(1900, datetime.now().year, inclusive="both")]

    silver_path = SILVER_DIR / "movies_metadata_silver.parquet"
    df.to_parquet(silver_path, index=False)
    print("Silver salvo em:", silver_path.resolve(), "→", df.shape)
else:
    print("Bronze não encontrado:", bronze_path)



## 4. Gold — Datasets finais
Seleção de colunas úteis para análises/modelos e particionamento simples.


In [ ]:

silver_path = SILVER_DIR / "movies_metadata_silver.parquet"
if silver_path.exists():
    df = pd.read_parquet(silver_path)
    cols_core = [
        "id_numeric","title_clean","original_language","release_date","release_year",
        "runtime","popularity","vote_average","vote_count","budget","revenue","roi",
        "genres_str","years_since_release","vote_density"
    ]
    df_gold = df[[c for c in cols_core if c in df.columns]].copy()
    out_path = GOLD_DIR / "movies_metadata_gold.parquet"
    df_gold.to_parquet(out_path, index=False)
    print("Gold salvo em:", out_path.resolve(), "→", df_gold.shape)
else:
    print("Silver não encontrado:", silver_path)



## 5. EDA Rápida
Alguns gráficos simples para entender distribuição de notas, relação orçamento×receita e gêneros populares.


In [ ]:

gold_path = GOLD_DIR / "movies_metadata_gold.parquet"
if gold_path.exists():
    dfg = pd.read_parquet(gold_path)

    # Histograma de notas
    plt.figure()
    dfg["vote_average"].dropna().plot(kind="hist", bins=20)
    plt.title("Distribuição de 'vote_average'")
    plt.xlabel("vote_average")
    plt.ylabel("freq")
    plt.show()

    # Dispersão orçamento vs receita (amostra para não poluir)
    plt.figure()
    sample = dfg.dropna(subset=["budget","revenue"]).sample(min(3000, len(dfg)), random_state=42)
    plt.scatter(sample["budget"], sample["revenue"], alpha=0.3)
    plt.title("Orçamento vs Receita (amostra)")
    plt.xlabel("budget")
    plt.ylabel("revenue")
    plt.show()

    # Top 10 gêneros por média de nota (simples, por ocorrência)
    if "genres_str" in dfg.columns:
        # expandir gêneros
        genre_rows = []
        for _, row in dfg[["genres_str","vote_average"]].dropna().iterrows():
            for g in [x.strip() for x in row["genres_str"].split(",") if x.strip()]:
                genre_rows.append((g, row["vote_average"]))
        if genre_rows:
            gdf = pd.DataFrame(genre_rows, columns=["genre","vote_average"])
            top = (gdf.groupby("genre")["vote_average"]
                      .mean()
                      .sort_values(ascending=False)
                      .head(10))
            plt.figure()
            top.plot(kind="bar")
            plt.title("Top 10 gêneros por média de nota")
            plt.xlabel("gênero")
            plt.ylabel("média de vote_average")
            plt.xticks(rotation=45, ha="right")
            plt.tight_layout()
            plt.show()
else:
    print("Gold não encontrado:", gold_path)



## 6. Baseline de Modelagem
Como baseline simples, vamos prever `vote_average` a partir de algumas features numéricas.
> **Observação:** Este modelo é apenas um ponto de partida para cumprir os requisitos e gerar um insumo analítico.


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression

gold_path = GOLD_DIR / "movies_metadata_gold.parquet"
if gold_path.exists():
    dfm = pd.read_parquet(gold_path).dropna(subset=["vote_average"])
    features = ["runtime","popularity","vote_count","budget","revenue","years_since_release","vote_density"]
    X = dfm[features].fillna(0)
    y = dfm["vote_average"]
    if len(X) > 0 and len(y) > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        pred = lr.predict(X_test)
        print("Baseline LinearRegression")
        print("R2:", round(r2_score(y_test, pred), 4))
        print("MAE:", round(mean_absolute_error(y_test, pred), 4))
    else:
        print("Dados insuficientes após preparação.")
else:
    print("Gold não encontrado:", gold_path)



## 7. Destino (consumo)
- Os datasets finais estão em `/dados/gold` (formato Parquet).
- Gráficos e tabelas podem ser salvos como imagens/CSVs na mesma pasta para consumo por dashboards.


In [ ]:

# Exemplo: salvar resumo estatístico e amostra dos dados gold
gold_path = GOLD_DIR / "movies_metadata_gold.parquet"
if gold_path.exists():
    dfg = pd.read_parquet(gold_path)
    dfg.describe(include="all").to_csv(GOLD_DIR / "gold_describe.csv")
    dfg.sample(min(1000, len(dfg)), random_state=42).to_csv(GOLD_DIR / "gold_sample.csv", index=False)
    print("Arquivos salvos em:", GOLD_DIR.resolve())



## (Opcional) 8. Simulação de *Streaming*
Simular ingestão em micro-lotes lendo o CSV linha a linha e gravando *chunks* em `/dados/bronze/stream/`.


In [ ]:

# Exemplo simples (descomente para testar com arquivo pequeno)
# stream_dir = BRONZE_DIR / "stream"
# stream_dir.mkdir(parents=True, exist_ok=True)
# chunk_size = 1000
# if raw_csv.exists():
#     reader = pd.read_csv(raw_csv, chunksize=chunk_size, low_memory=False)
#     for i, chunk in enumerate(reader):
#         path = stream_dir / f"chunk_{i:04d}.parquet"
#         chunk.to_parquet(path, index=False)
#         print("gravou:", path)
# else:
#     print("Arquivo não encontrado para stream:", raw_csv)



---
### Checklist AV1 (preencher no README)
- [ ] Ingestão: Em progresso / Finalizado / Pendente  
- [ ] Armazenamento: Em progresso / Finalizado / Pendente  
- [ ] Transformação: Em progresso / Finalizado / Pendente  
- [ ] Demonstração técnica: pronto para apresentação  

> Gerado em: 2025-10-06 13:46:46
